# Tutorial 6: Work with Tables

This Tutorial demonstrates how

## SIR 3S Installation

In [101]:
SIR3S_SIRGRAF_DIR = r"C:\3S\SIR 3S\SirGraf-90-15-00-21_Quebec-Upd2" #change to local path

## Imports

Note: The SIR 3S Toolkit requires the Sir3S_Toolkit.dll included in SIR 3S installations (version Quebec and higher).

In [102]:
import sir3stoolkit

The core of sir3stoolkit is a Python wrapper around basic functionality of SIR 3S, offering a low-level access to the creation, modification and simulation of SIR 3S models. In the future pure python subpackages may be added.

In [103]:
from sir3stoolkit.core import wrapper

In [104]:
sir3stoolkit

<module 'sir3stoolkit' from 'C:\\Users\\aUsername\\3S\\sir3stoolkit\\src\\sir3stoolkit\\__init__.py'>

The [wrapper package](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.Initialize_Toolkit) has to be initialized with reference to a SIR 3S (SirGraf) installation.

In [105]:
wrapper.Initialize_Toolkit(SIR3S_SIRGRAF_DIR)

## Initialization

The SIR 3S Toolkit contains two classes: [SIR3S_Model](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model) (model and data) and [SIR3S_View](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_View) (depiction in SIR Graf). All SIR 3S Toolkit functionality is accessed via the methods of these classes.

In [106]:
s3s = wrapper.SIR3S_Model()

Initialization complete


In [107]:
s3s_view = wrapper.SIR3S_View()

Initialization complete


# Open Model

In [108]:
dbFilePath=r"C:\Users\aUsername\3S\PT3S\PT3S\Examples\Example5.db3"

In [109]:
dbFilePath=r"Toolkit_Tutorial6_Model.db3"

In [110]:
s3s.OpenModel(dbName=dbFilePath,
              providerType=s3s.ProviderTypes.SQLite,
              Mid="M-1-0-1",
              saveCurrentlyOpenModel=False,
              namedInstance="",
              userID="",
              password="")

Model is open for further operation


In [111]:
print(s3s.GetNetworkType()) # to check that the correct model is responsive, model we are trying to open was created with type Undefined

NetworkType.Water


## GetTableRows()

We can use the [GetTableRows()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.GetTableRows) function to obtain the tks of rows belonging to the table with a given tk.

In [112]:
for tk in s3s.GetTksofElementType(s3s.ObjectTypes.PipeTable):
    print(tk + ": " + s3s.GetValue(tk, "Name")[0])


5464993648735182081: STDROHR
5084212600034677044: AZ_PN10
4805611686096369081: StB_PN20
5266199647394782751: GUSS_K8
5015394485292264252: GUSS_K8Z
4879264831531644476: DN2460G
4702669660282998254: DN2460GZ
5584209084918079490: ST-V4
4747010066358051293: GFKUPGFPN10
5497475942106574922: PE100SDR11
4947305676931088023: PE100SDR17
5544640728947443469: PP_PN10
4740109886480699434: PVCPN10A
4743437019149053639: PVCPN10T


Let's focus on the STDROHR table

In [113]:
tk_STDROHR_table = "5464993648735182081"

In [114]:
rows = s3s.GetTableRows(tablePkTk=tk_STDROHR_table)

In [115]:
rows[0]

<System.String[] object at 0x00000217EEDE7980>

We get a .NET array of strings returned, we need to transform it into a python list.

In [116]:
row_tks = list(rows[0])  

Now, we have the tks of all rows of this table.

We can reconstruct the table via pandas as follows.

In [117]:
import pandas as pd

In [118]:
properties_rows=s3s.GetPropertiesofElementType(s3s.ObjectTypes.PipeTable_Row)

In [119]:
data = {
    tk: {prop: s3s.GetValue(tk, prop)[0] for prop in properties_rows}
    for tk in row_tks
}

In [120]:
df = pd.DataFrame.from_dict(data, orient='index')

In [121]:
df.head(5)

,Name,Fk,Dn,Di,Da,S,Wsteig,Wtiefe,Kt,Pn,Ausfallzeit,Reparatur,Rehabilitation,Tk,Pk,InVariant
4763523255060525502,DN 3,5464993648735182081,3,3,0,0,0,0,0,0,0,0,0,4763523255060525502,4763523255060525502,False
4985118461854533500,DN 4,5464993648735182081,4,4,0,0,0,0,0,0,0,0,0,4985118461854533500,4985118461854533500,False
5240628392234716908,DN 5,5464993648735182081,5,5,0,0,0,0,0,0,0,0,0,5240628392234716908,5240628392234716908,False
5735560888842799373,DN 6,5464993648735182081,6,6,0,0,0,0,0,0,0,0,0,5735560888842799373,5735560888842799373,False
4891609214500630565,DN 8,5464993648735182081,8,8,0,0,0,0,0,0,0,0,0,4891609214500630565,4891609214500630565,False


## AddTableRow()

We can use the [AddTableRow()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.AddTableRow) function to add a new row to the bottom of any table. 

In [122]:
new_table_row_tk = s3s.AddTableRow(tk_STDROHR_table)[0]

Row is added to the table with Tk: 5339161840932807029


Now we can use the already known [SetValue()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.SetValue) function to propagate the row with the necessary values.

In [123]:
s3s.GetPropertiesofElementType(s3s.ObjectTypes.PipeTable_Row)

['Name',
 'Fk',
 'Dn',
 'Di',
 'Da',
 'S',
 'Wsteig',
 'Wtiefe',
 'Kt',
 'Pn',
 'Ausfallzeit',
 'Reparatur',
 'Rehabilitation',
 'Tk',
 'Pk',
 'InVariant']

In [124]:
s3s.SetValue(new_table_row_tk, "Name", "DN 60")

Value is set


In [125]:
s3s.SetValue(new_table_row_tk, "Fk", "5207173582970272729")

Value is set


In [126]:
s3s.SetValue(new_table_row_tk, "Dn", "60")

Value is set


... and so on. Then save your changes.

In [127]:
#s3s.SaveChanges()

# WORK IN PRORGRESS: Sollwerttabellen

In [128]:
s3s.CloseModel(False)

True

In [132]:
dbFilePath=r"C:\Users\aUsername\3S\PT3S\PT3S\Examples\Example5.db3"

In [133]:
s3s.OpenModel(dbName=dbFilePath,
              providerType=s3s.ProviderTypes.SQLite,
              Mid="M-1-0-1",
              saveCurrentlyOpenModel=False,
              namedInstance="",
              userID="",
              password="")

Model is open for further operation


In [140]:
for tk in s3s.GetTksofElementType(s3s.ObjectTypes.MeasuredVariableTable):
    print(tk + ": " + s3s.GetValue(tk, "Name")[0])


4791958075034637588: handA
4716831806719265523: handB
5392006316499374132: wDH_RD_A
5057312816549531817: wTRSP
5153463029238392158: zLeck_VL
5001286428289125470: wDH_BA_A
4899737465956524526: SWVT
4624595674627180198: zLeck2
5315880859498738582: zLeck1
4752716540990910920: zLeck3
5037735657984644647: wDH_MD_A
4735319280169143557: zLeckMenge
5736925379101492692: wNB
5637532099392751986: wAutoB
4673315691931193810: wLast
5297827003965178187: wAutoA
4802336486945290037: wNA
4855598783561116318: zLeck_RL
5709867941939642203: wDH_RD_MD_A
5191523927382509291: zLeck4
5597720830879325124: zLeck5
5472594995058836608: zLeck6
5441042976828796358: zLeck7
5500180644347508185: zLeck8
5296078303385624388: zLeck9
4760880579110230581: TS9RL2
4824264679525335382: TS9VL2
5035100653120857096: TS9VL1
5086802958757995067: TS9RL1
4916486740814847304: TS01_03_01V
4833437403880726369: TS01_03_01R
5127910863327281205: TS02_03_02V
5531709989643944492: TS02_03_02R
4992917373283800876: TS03_04_03V
5302187966003527

In [153]:
rows = s3s.GetTableRows(tablePkTk="4735319280169143557")

In [154]:
row_tks = list(rows[0])  

In [159]:
properties_rows=s3s.GetPropertiesofElementType(s3s.ObjectTypes.MeasuredVariableTable_Row)

In [160]:
data = {
    tk: {prop: s3s.GetValue(tk, prop)[0] for prop in properties_rows}
    for tk in row_tks
}

In [161]:
df = pd.DataFrame.from_dict(data, orient='index')

In [162]:
df.head(5)

,Name,Fk,Zeit,Zeitstempel,W,Pk,InVariant
5001792674114123652,SWVTROWT pk=5001792674114123652,4735319280169143557,0,,0,5001792674114123652,False
5163058371591509468,SWVTROWT pk=5163058371591509468,4735319280169143557,120,,0,5163058371591509468,False
5097921829828618820,SWVTROWT pk=5097921829828618820,4735319280169143557,1320,,0,5097921829828618820,False
5366625369367435218,SWVTROWT pk=5366625369367435218,4735319280169143557,3600,,100,5366625369367435218,False
4953891471038282180,SWVTROWT pk=4953891471038282180,4735319280169143557,6600,,0,4953891471038282180,False
